<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [7]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

## Setup

In [2]:
import datetime
import logging
import os
import subprocess
import time
import google.cloud.storage as gcs

from googleapiclient import discovery
from googleapiclient import errors
from googleapiclient import http

BUCKET="dataflow-jlewi-cloud-ml"
PROJECT="dataflow-jlewi"
logging.getLogger().setLevel(logging.INFO)

cloudml = discovery.build(
      'ml',
      'v1',
      requestBuilder=http.HttpRequest)


## Create Training Data

In [3]:
# Make sure you're logged in and have credentials
command = ["python", "-m", "cmle_example.create_training_data",
           "--output_path=gs://{0}/running_average/inputs.tfrecord".format(BUCKET)]

output = subprocess.check_output(command,stderr=subprocess.STDOUT)
print output

INFO:root:Creating data
INFO:root:Creating example: 1
INFO:root:Creating example: 2
INFO:root:Creating example: 3
INFO:root:Creating example: 4
INFO:root:Creating example: 5
INFO:root:Creating example: 6
INFO:root:Creating example: 7
INFO:root:Creating example: 8
INFO:root:Creating example: 9
INFO:root:Creating example: 10
INFO:root:Creating example: 11
INFO:root:Creating example: 12
INFO:root:Creating example: 13
INFO:root:Creating example: 14
INFO:root:Creating example: 15
INFO:root:Creating example: 16
INFO:root:Creating example: 17
INFO:root:Creating example: 18
INFO:root:Creating example: 19
INFO:root:Creating example: 20
INFO:root:Creating example: 21
INFO:root:Creating example: 22
INFO:root:Creating example: 23
INFO:root:Creating example: 24
INFO:root:Creating example: 25
INFO:root:Creating example: 26
INFO:root:Creating example: 27
INFO:root:Creating example: 28
INFO:root:Creating example: 29
INFO:root:Creating example: 30
INFO:root:Creating example: 31
INFO:root:Creating examp

## Submit Training Job

In [5]:
from cmle_example import submit_training_job
reload(submit_training_job)
now = datetime.datetime.now()
datestamp = now.strftime("%Y%m%d_%H%M%S")
job_name="cmle_example_{0}".format(datestamp)
output_path = "gs://{0}/running_average/{1}/output/results".format(BUCKET, datestamp)
main_args = ["--train_data_path=gs://{0}/running_average/inputs.tfrecord".format(BUCKET)]
final_job = submit_training_job.submit_job(job_name, main_args, output_path, PROJECT)

INFO:root:Setup file: /content/datalab/home/git_cmle_simple/setup.py
INFO:root:Found packages: ['/tmp/tmpfaaWGQ/cmle_example-0.1.1.tar.gz']
INFO:oauth2client.client:Attempting refresh to obtain initial access_token
INFO:oauth2client.client:Refreshing access_token
INFO:root:Copying /tmp/tmpfaaWGQ/cmle_example-0.1.1.tar.gz to gs://dataflow-jlewi-cloud-ml/running_average/20170827_224405/output/results/staging/cmle_example-0.1.1.tar.gz
INFO:googleapiclient.discovery:URL being requested: POST https://ml.googleapis.com/v1/projects/dataflow-jlewi/jobs?alt=json
INFO:oauth2client.client:Attempting refresh to obtain initial access_token
INFO:oauth2client.client:Refreshing access_token
INFO:root:Set response:
{
  "createTime": "2017-08-27T22:44:13Z",
  "jobId": "cmle_example_20170827_224405",
  "state": "QUEUED",
  "trainingInput": {
    "args": [
      "--train_data_path=gs://dataflow-jlewi-cloud-ml/running_average/inputs.tfrecord",
      "--output_path=gs://dataflow-jlewi-cloud-ml/running_avera

## Save Model For Serving

In [8]:
from cmle_example import build_saved_model
reload(build_saved_model)
saved_model = "gs://{0}/cmle_example/{1}/saved_model".format(BUCKET, datestamp)
build_saved_model.save_model(output_path, saved_model)

INFO:root:Exporting the model to gs://dataflow-jlewi-cloud-ml/cmle_example/20170827_224405/saved_model
INFO:root:Restoring weights from: gs://dataflow-jlewi-cloud-ml/running_average/20170827_224405/output/results/save_dir-5


INFO:tensorflow:Restoring parameters from gs://dataflow-jlewi-cloud-ml/running_average/20170827_224405/output/results/save_dir-5


INFO:tensorflow:Restoring parameters from gs://dataflow-jlewi-cloud-ml/running_average/20170827_224405/output/results/save_dir-5
INFO:root:inputs: ParseExample/ParseExample:0


INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: gs://dataflow-jlewi-cloud-ml/cmle_example/20170827_224405/saved_model/saved_model.pb


INFO:tensorflow:SavedModel written to: gs://dataflow-jlewi-cloud-ml/cmle_example/20170827_224405/saved_model/saved_model.pb


## Deploy the model

In [10]:
from cmle_example import deploy_model
reload(deploy_model)
endpoint = None
model_name = "simple_cmle"
model_path = saved_model

now = datetime.datetime.now()
datestamp = now.strftime("%Y%m%d_%H%M%S")
model_version = 'v' + datestamp

deploy_model.deploy(PROJECT, model_path, model_name, model_version)


INFO:googleapiclient.discovery:URL being requested: POST https://ml.googleapis.com/v1/projects/dataflow-jlewi/models?alt=json
INFO:root:Model simple_cmle already exists.
INFO:googleapiclient.discovery:URL being requested: POST https://ml.googleapis.com/v1/projects/dataflow-jlewi/models/simple_cmle/versions?alt=json
INFO:root:Set response:
{
  "metadata": {
    "@type": "type.googleapis.com/google.cloud.ml.v1.OperationMetadata",
    "createTime": "2017-08-27T22:47:28Z",
    "modelName": "projects/dataflow-jlewi/models/simple_cmle",
    "operationType": "CREATE_VERSION",
    "version": {
      "createTime": "2017-08-27T22:47:28Z",
      "deploymentUri": "gs://dataflow-jlewi-cloud-ml/cmle_example/20170827_224405/saved_model",
      "etag": "sDkb0ns6t8c=",
      "name": "projects/dataflow-jlewi/models/simple_cmle/versions/v20170827_224726",
      "state": "CREATING"
    }
  },
  "name": "projects/dataflow-jlewi/operations/create_simple_cmle_v20170827_224726-1503874048610"
}


{u'metadata': {u'@type': u'type.googleapis.com/google.cloud.ml.v1.OperationMetadata',
  u'createTime': u'2017-08-27T22:47:28Z',
  u'modelName': u'projects/dataflow-jlewi/models/simple_cmle',
  u'operationType': u'CREATE_VERSION',
  u'version': {u'createTime': u'2017-08-27T22:47:28Z',
   u'deploymentUri': u'gs://dataflow-jlewi-cloud-ml/cmle_example/20170827_224405/saved_model',
   u'etag': u'sDkb0ns6t8c=',
   u'name': u'projects/dataflow-jlewi/models/simple_cmle/versions/v20170827_224726',
   u'state': u'CREATING'}},
 u'name': u'projects/dataflow-jlewi/operations/create_simple_cmle_v20170827_224726-1503874048610'}

## Send some predictions

In [12]:
from cmle_example import util
projects = cloudml.projects()

body = {
  'instances' : [
    {
      'inputs': [10],
    },
    {
      'inputs': [15],
    }
  ],
}
request = projects.predict(name='projects/{0}/models/{1}/versions/{2}'.format(PROJECT, model_name, model_version),
                           body=body)

response = request.execute()
logging.info('Set response:\n%s', util.PrettyFormat(response))

INFO:googleapiclient.discovery:URL being requested: POST https://ml.googleapis.com/v1/projects/dataflow-jlewi/models/simple_cmle/versions/v20170827_224726:predict?alt=json


HttpError: <HttpError 404 when requesting https://ml.googleapis.com/v1/projects/dataflow-jlewi/models/simple_cmle/versions/v20170827_224726:predict?alt=json returned "Field: name Error: Online prediction is unavailable for this version. Please verify that CreateVersion has completed successfully.">